In [ ]:
import torch as torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from DataRead import OurData
import traceback

from kan import KAN, create_dataset

torch.set_default_dtype(torch.float32)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = 28*28 # from 28*28

In [10]:
import struct
torch.set_default_dtype(torch.float32)

def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, dtype, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for _ in range(dims))
        return torch.tensor(np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)).to(torch.float32)

class OurData:
    def __init__(self):
        self.ourdataset = {}
        train_images = read_idx('/workspaces/KAN-Network/Dataset/train-images-idx3-ubyte/train-images-idx3-ubyte') #contains the training data, each data is the binary representation of an image as per the MNIST dataset
        self.ourdataset['train_input'] = (train_images).view(-1, 28*28) #reshapes the data into a 2D array, each row is an image
        test_images = read_idx('/workspaces/KAN-Network/Dataset/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte') #contains the testing data, same format as train_input
        self.ourdataset['test_input'] = (test_images).view(-1, 28*28) #reshapes the data into a 2D array, each row is an image
        self.ourdataset['train_label'] = read_idx('/workspaces/KAN-Network/Dataset/train-labels-idx1-ubyte/train-labels-idx1-ubyte').unsqueeze(1) #contains the labels for the training data
        self.ourdataset['test_label'] = read_idx('/workspaces/KAN-Network/Dataset/t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte').unsqueeze(1) #contains the labels for the testing data
        #print("These are our training inputs and labels")
    def __getitem__(self):
        return self.ourdataset
    def __getitem__(self, key):
        return self.ourdataset[key]

In [ ]:
'''
initialize model
- refer to MultKAN.py for more information
    width: number of neurons in each layer, in order from input to output
    k: order of the spline
    seed: random seed
    grid: grid intervals/grid points (affects the accuracy of the splines/learnable functions)
'''
model = KAN(width=[input_size, 5, 1], grid=5, k=3, seed=0, device=device)

checkpoint directory created: ./model
saving model version 0.0


In [14]:
#Our dataset
data = OurData()
ourdata = {}
for key in data.ourdataset:
    ourdata[key] = (data.ourdataset[key])[:100] #only get the first 100 data points for now

model(ourdata['train_input']) #forward pass of the model
print("model pass complete")
#model.plot() #plots the model, avoid doing this since it will plot functions for all the neurons(and we have a lot since we are dealing with images)

model pass complete


In [15]:
#fits the model to the dataset
try:
    #currently doesn't work since all the coefficients are uninitialized???
    #the coefficients are uninitialized due to problems in the initial forward pass
    model.fit(ourdata, opt="LBFGS", steps=50, lamb=0.001) #values from the basic example in the documentation
#model.plot() 
except Exception as e:
    print(e)
    traceback.print_exc()

| train_loss: 3.80e-02 | test_loss: 2.14e+01 | reg: 6.44e+01 | : 100%|█| 50/50 [00:44<00:00,  1.13it

saving model version 0.2


In [13]:
#todo: change the model plot function?
#model = model.prune()
#model.plot()